## 載入模型

In [2284]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [2285]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [2286]:
# 載入模型與標準化器
base_model = load_model('Model_SAVE/4 mc AP RSSI FTM StdDev 2024-12-14 data train/DNN_best_model4mcAP.h5')

scaler = joblib.load('Model_SAVE/4 mc AP RSSI FTM StdDev 2024-12-14 data train/scaler_4mcAP.pkl')

In [2287]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 64)                832       
                                                                 
 dense_45 (Dense)            (None, 128)               8320      
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 49)                3185      
                                                                 
Total params: 20,593
Trainable params: 3,185
Non-trainable params: 17,408
_________________________________________________________________


In [2288]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [2289]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
        'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
        'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [2290]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2024_12_21_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2024_12_21"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-11,444.0,6841.0,7353.0,5116.0,-60.0,-71.0,-50.0,-62.0,809.0,705.0,210.0,996.0
1,1-11,366.0,6841.0,7236.0,5242.0,-59.0,-71.0,-54.0,-66.0,808.0,766.0,719.0,420.0
2,1-11,288.0,8569.0,7217.0,4738.0,-58.0,-71.0,-56.0,-64.0,506.0,531.0,131.0,2157.0
3,1-11,366.0,6958.0,7327.0,6854.0,-59.0,-71.0,-52.0,-58.0,842.0,680.0,348.0,1181.0
4,1-11,327.0,6899.0,7275.0,6887.0,-59.0,-71.0,-54.0,-62.0,907.0,706.0,451.0,741.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18860,7-1,6284.0,2358.0,7783.0,594.0,-73.0,-74.0,-58.0,-58.0,849.0,991.0,933.0,576.0
18861,7-1,6372.0,2280.0,7627.0,516.0,-66.0,-75.0,-60.0,-59.0,1409.0,909.0,735.0,679.0
18862,7-1,6430.0,2202.0,7705.0,360.0,-73.0,-76.0,-60.0,-59.0,869.0,924.0,731.0,781.0
18863,7-1,6313.0,2280.0,7705.0,516.0,-73.0,-74.0,-60.0,-59.0,866.0,1198.0,750.0,598.0


In [2291]:
# 資料前處理 (一): 刪除前後n筆資料
n = 10
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 1112
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3960.0,6265.0,13105.0,-342.0,-63.0,-74.0,-65.0,-49.0,1544.0,741.0,949.0,1243.0
1,1-1,1,3999.0,6694.0,13457.0,-381.0,-63.0,-74.0,-67.0,-52.0,1511.0,508.0,1696.0,1297.0
2,1-1,2,4038.0,6460.0,13281.0,-264.0,-64.0,-75.0,-69.0,-52.0,1408.0,546.0,999.0,1244.0
3,1-1,3,4194.0,6421.0,13960.0,-381.0,-63.0,-74.0,-63.0,-48.0,1005.0,620.0,948.0,1466.0
4,1-1,4,4189.0,6460.0,13667.0,-381.0,-58.0,-73.0,-63.0,-54.0,995.0,553.0,860.0,1565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,9-11,17880,1655.0,8569.0,2549.0,4825.0,-55.0,-67.0,-52.0,-67.0,354.0,70.0,606.0,491.0
17881,9-11,17881,1889.0,8452.0,2549.0,4998.0,-46.0,-67.0,-51.0,-70.0,509.0,131.0,493.0,86.0
17882,9-11,17882,1928.0,8803.0,2588.0,4998.0,-55.0,-68.0,-52.0,-71.0,215.0,211.0,444.0,87.0
17883,9-11,17883,1968.0,8481.0,2588.0,5086.0,-55.0,-67.0,-51.0,-70.0,242.0,168.0,460.0,125.0


In [2292]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [2293]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

y_test_numeric

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


0        10
1        10
2        10
3        10
4        10
         ..
17880    32
17881    32
17882    32
17883    32
17884    32
Name: Label, Length: 17885, dtype: int64

In [2294]:
# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])

# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [2295]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

### 重新訓練model

In [2296]:
# 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
from sklearn.model_selection import StratifiedShuffleSplit

# 設定每個類別的固定樣本數
N_train = 40 # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 0.5  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

print((data['label'] == 10).sum())  # 直接計算 True 的數量


365


In [2297]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,0.500544,0.353457,2.375205,-1.250919,1.015252,-0.024415,0.491255,0.637203,-0.547117,-1.242229,-1.294873,1.345117,10
1,0.517006,0.500142,2.508842,-1.267033,0.962320,-0.380482,1.971711,0.735578,-0.547117,-1.242229,-1.674492,0.880167,10
2,0.533468,0.420132,2.442023,-1.218690,0.797106,-0.322411,0.590348,0.639024,-0.760283,-1.425137,-2.054111,0.880167,10
3,0.599316,0.406797,2.699805,-1.267033,0.150689,-0.209325,0.489273,1.043458,-0.547117,-1.242229,-0.915254,1.500100,10
4,0.597205,0.420132,2.588568,-1.267033,0.134649,-0.311713,0.314868,1.223814,0.518718,-1.059321,-0.915254,0.570201,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,-0.472402,1.141245,-1.632377,0.884007,-0.893524,-1.049826,-0.188526,-0.732771,1.158218,0.038127,1.172651,-1.444580,32
17881,-0.373630,1.101241,-1.632377,0.955488,-0.644902,-0.956607,-0.412478,-1.470590,3.076720,0.038127,1.362461,-1.909529,32
17882,-0.357168,1.221255,-1.617570,0.955488,-1.116482,-0.834352,-0.509589,-1.468768,1.158218,-0.144781,1.172651,-2.064512,32
17883,-0.340284,1.111156,-1.617570,0.991848,-1.073174,-0.900064,-0.477879,-1.399541,1.158218,0.038127,1.362461,-1.909529,32


In [2298]:
# 儲存訓練集
train_data = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
# print(train_data.index)  # 查看 train_data 的索引

print((train_data['label'] == 10).sum())  # 計算 label 為 10 的數量


40


In [2299]:
remaining_data = data.drop(train_data.index)
print(remaining_data['label'].value_counts().get(10, 0))  # 應該是 348


325


In [2300]:
# 取得剩餘資料的 features 和 labels
X_remaining = remaining_data.drop(columns=['label'])
y_remaining = remaining_data['label']

print(y_remaining.value_counts().get(10, 0))

325


In [2301]:
# 使用 StratifiedShuffleSplit 來確保驗證集與測試集的類別比例一致
sss = StratifiedShuffleSplit(n_splits=1, test_size=test_val_ratio, random_state=42)
for val_index, test_index in sss.split(X_remaining, y_remaining):
    X_val, X_test = X_remaining.iloc[val_index], X_remaining.iloc[test_index]
    y_val, y_test = y_remaining.iloc[val_index], y_remaining.iloc[test_index]

# 轉換回 NumPy 陣列
X_train_small, y_train_small = train_data.drop(columns=['label']).values, train_data['label'].values
X_val, y_val = X_val.values, y_val.values
X_test, y_test = X_test.values, y_test.values

# 確認切分後的數據數量
print(f"Training set: {len(X_train_small)} samples, {len(np.unique(y_train_small))} unique labels")
print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

Training set: 1960 samples, 49 unique labels
Validation set: 7962 samples, 49 unique labels
Test set: 7963 samples, 49 unique labels


In [2302]:
# # 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
# X_train_small, X_unused, y_train_small, y_unused = train_test_split(
#     X_test_scaled, y_test_numeric, test_size=0.99, random_state=42, stratify=y_test_numeric
# )
# # 再切出一半作為validation set and test set
# X_val, X_test, y_val, y_test = train_test_split(
#     X_unused, y_unused, test_size=0.5, random_state=42, stratify=y_unused
# )

In [2303]:
# 檢查各組資料比例
print(f"Training set: {len(X_train_small)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

Training set: 1960 samples
Validation set: 7962 samples
Test set: 7963 samples


In [2304]:
import pandas as pd

# 計算每個 Set 內各 Label 的資料數量
train_label_counts = pd.Series(y_train_small).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# # 印出統計結果
# print("Training Set - Label Counts:")
# print(train_label_counts)

# print("\nValidation Set - Label Counts:")
# print(val_label_counts)

# print("\nTest Set - Label Counts:")
# print(test_label_counts)

# 確保所有 Labels 都有出現在三個 Set 裡
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

# 用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）
label_distribution = label_distribution.fillna(0).astype(int)

from IPython.display import display
display(label_distribution)


,Training Set,Validation Set,Test Set
0,40,162,163
1,40,162,163
2,40,162,163
3,40,163,162
4,40,163,162
5,40,162,163
6,40,163,162
7,40,162,163
8,40,163,162
9,40,163,162


In [2305]:
import time
# 記錄開始時間
start_time = time.time()

In [2306]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [2307]:
base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=32, callbacks=[early_stop])

Epoch 1/10000
62/62 [==============================] - 1s 6ms/step - loss: 4.6847 - accuracy: 0.5015 - val_loss: 3.4209 - val_accuracy: 0.5369
Epoch 2/10000
62/62 [==============================] - 0s 2ms/step - loss: 2.8522 - accuracy: 0.5781 - val_loss: 2.1153 - val_accuracy: 0.6179
Epoch 3/10000
62/62 [==============================] - 0s 4ms/step - loss: 1.8555 - accuracy: 0.6755 - val_loss: 1.3697 - val_accuracy: 0.7138
Epoch 4/10000
62/62 [==============================] - 0s 3ms/step - loss: 1.2554 - accuracy: 0.7546 - val_loss: 0.9484 - val_accuracy: 0.7840
Epoch 5/10000
62/62 [==============================] - 0s 2ms/step - loss: 0.8928 - accuracy: 0.8184 - val_loss: 0.7179 - val_accuracy: 0.8365
Epoch 6/10000
62/62 [==============================] - 0s 3ms/step - loss: 0.6950 - accuracy: 0.8566 - val_loss: 0.5804 - val_accuracy: 0.8665
Epoch 7/10000
62/62 [==============================] - 0s 2ms/step - loss: 0.5639 - accuracy: 0.8872 - val_loss: 0.4908 - val_accuracy: 0.8857

In [2308]:
# 記錄結束時間
end_time = time.time()

# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：24.58 秒


In [2309]:
# 使用剩餘的 90% 測試資料進行模型評估
loss, accuracy = base_model.evaluate(X_test, y_test)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

249/249 [==============================] - 0s 919us/step - loss: 0.2066 - accuracy: 0.9851
Evaluation on 90% unused data - Loss: 0.2066, Accuracy: 0.9851


In [2310]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [2311]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

559/559 [==============================] - 0s 633us/step


['1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '2-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',


In [2312]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,3960.0,6265.0,13105.0,-342.0,-63.0,-74.0,-65.0,-49.0,1544.0,741.0,949.0,1243.0
1,1-1,1,3999.0,6694.0,13457.0,-381.0,-63.0,-74.0,-67.0,-52.0,1511.0,508.0,1696.0,1297.0
2,1-1,2,4038.0,6460.0,13281.0,-264.0,-64.0,-75.0,-69.0,-52.0,1408.0,546.0,999.0,1244.0
3,1-1,3,4194.0,6421.0,13960.0,-381.0,-63.0,-74.0,-63.0,-48.0,1005.0,620.0,948.0,1466.0
4,1-1,4,4189.0,6460.0,13667.0,-381.0,-58.0,-73.0,-63.0,-54.0,995.0,553.0,860.0,1565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17880,9-11,17880,1655.0,8569.0,2549.0,4825.0,-55.0,-67.0,-52.0,-67.0,354.0,70.0,606.0,491.0
17881,9-11,17881,1889.0,8452.0,2549.0,4998.0,-46.0,-67.0,-51.0,-70.0,509.0,131.0,493.0,86.0
17882,9-11,17882,1928.0,8803.0,2588.0,4998.0,-55.0,-68.0,-52.0,-71.0,215.0,211.0,444.0,87.0
17883,9-11,17883,1968.0,8481.0,2588.0,5086.0,-55.0,-67.0,-51.0,-70.0,242.0,168.0,460.0,125.0


In [2313]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2024_12_21.json

Test Data Mean MDE: 0.0210 meters


In [2314]:
base_model.save(f"DNN_best_model4mcAP_1week_40dataperRP.h5")